In [5]:
from decimal import Decimal, getcontext
import itertools
import pandas as pd

getcontext().prec = 12

values_list = [
    Decimal('45.64'),
    Decimal('50.98'),
    Decimal('72.00'),
    Decimal('92.00'),
    Decimal('147.35'),
    Decimal('184.19'),
    Decimal('190.59'),
    Decimal('196.88'),
    Decimal('201.02'),
    Decimal('251.00'),
    Decimal('256.44'),
    Decimal('286.69'),
    Decimal('317.82'),
    Decimal('349.48'),
    Decimal('359.70'),
    Decimal('368.62'),
    Decimal('393.90'),
    Decimal('404.45'),
    Decimal('409.50'),
    Decimal('460.00'),
    Decimal('464.80'),
    Decimal('477.29'),
    Decimal('477.87'),
    Decimal('482.70'),
    Decimal('493.02'),
    Decimal('514.14'),
    Decimal('600.00'),
    Decimal('600.00'),
    Decimal('600.00'),
    Decimal('615.83'),
    Decimal('620.28'),
    Decimal('665.65'),
    Decimal('690.00'),
    Decimal('704.00'),
    Decimal('720.24'),
    Decimal('732.85'),
    Decimal('739.26'),
    Decimal('757.00'),
    Decimal('765.96'),
    Decimal('785.08'),
    Decimal('808.05'),
    Decimal('842.00'),
    Decimal('951.65'),
    Decimal('964.74'),
    Decimal('1007.40'),
    Decimal('1137.28'),
    Decimal('1320.51'),
    Decimal('1333.50'),
    Decimal('1542.18'),
    Decimal('1683.75'),
    Decimal('1795.00'),
    Decimal('1912.00'),
    Decimal('1932.77'),
    Decimal('2024.63'),
    Decimal('2183.40'),
    Decimal('2214.31'),
    Decimal('2296.85'),
    Decimal('2297.36'),
    Decimal('2310.96'),
    Decimal('2529.56'),
    Decimal('2551.63'),
    Decimal('3201.33'),
    Decimal('3441.04'),
    Decimal('3652.79'),
    Decimal('4123.31'),
    Decimal('4585.21'),
    Decimal('7701.51'),
    Decimal('9002.44')
]

target = Decimal('16645.85')

n = len(values_list)
solutions = []

# Iterate subsets; 2^~74 is huge. But number of items is about 74? Let's check n.
n

68

In [6]:
# Use integer cents DP to find up to 10 unique combinations summing to target.
from collections import deque

values_cents = [int((v * 100).to_integral_value()) for v in values_list]
target_cents = int((target * 100).to_integral_value())

# DP dict: soma -> (soma anterior, índice usado)
dp = {0: []}

found_solutions = []        # lista final de soluções únicas
seen_combinations = set()   # para evitar repetições

for idx, val in enumerate(values_cents):
    current_sums = list(dp.keys())
    for s in current_sums:
        new_s = s + val
        if new_s > target_cents:
            continue
        if new_s not in dp:
            dp[new_s] = (s, idx)
            if new_s == target_cents:
                # reconstruir solução (lista de índices)
                sol = []
                cur = new_s
                while cur != 0:
                    prev, used_idx = dp[cur]
                    sol.append(used_idx)
                    cur = prev
                sol = list(reversed(sol))

                # normalizar para evitar repetição (ordena índices)
                sol_key = tuple(sorted(sol))
                if sol_key not in seen_combinations:
                    seen_combinations.add(sol_key)
                    found_solutions.append(sol)

                if len(found_solutions) >= 1:  # parar após uma solução
                    break
    if len(found_solutions) >= 1:
        break

print(f"Total soluções únicas encontradas: {len(found_solutions)}")
for i, sol in enumerate(found_solutions, 1):
    valores = [values_list[j] for j in sol]
    print(f"Solução {i}: índices {sol}, \nvalores {valores}")

Total soluções únicas encontradas: 1
Solução 1: índices [0, 2, 3, 4, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40], 
valores [Decimal('45.64'), Decimal('72.00'), Decimal('92.00'), Decimal('147.35'), Decimal('201.02'), Decimal('251.00'), Decimal('256.44'), Decimal('286.69'), Decimal('349.48'), Decimal('359.70'), Decimal('368.62'), Decimal('393.90'), Decimal('404.45'), Decimal('409.50'), Decimal('460.00'), Decimal('464.80'), Decimal('477.29'), Decimal('477.87'), Decimal('482.70'), Decimal('493.02'), Decimal('514.14'), Decimal('600.00'), Decimal('600.00'), Decimal('600.00'), Decimal('615.83'), Decimal('620.28'), Decimal('665.65'), Decimal('690.00'), Decimal('704.00'), Decimal('720.24'), Decimal('732.85'), Decimal('739.26'), Decimal('757.00'), Decimal('785.08'), Decimal('808.05')]


In [ ]:

from decimal import Decimal

def find_combination(values_list, target, must_include=None, max_solutions=11):
    """
    Encontra até max_solutions combinações únicas de values_list que somam target.
    Se must_include for fornecido, todas as combinações devem conter esses valores.
    """
    values_cents = [int((v * 100).to_integral_value()) for v in values_list]
    target_cents = int((target * 100).to_integral_value())

    must_include_cents = None
    if must_include:
        must_include_cents = [int((v * 100).to_integral_value()) for v in must_include]

    dp = {0: None}
    found_solutions = []
    seen_combinations = set()

    for idx, val in enumerate(values_cents):
        current_sums = list(dp.keys())
        for s in current_sums:
            new_s = s + val
            if new_s > target_cents:
                continue
            if new_s not in dp:
                dp[new_s] = (s, idx)
                if new_s == target_cents:
                    # reconstruir solução (lista de índices)
                    sol = []
                    cur = new_s
                    while cur != 0:
                        prev, used_idx = dp[cur]
                        sol.append(used_idx)
                        cur = prev
                    sol = list(reversed(sol))

                    # normalizar para evitar repetição (ordena índices)
                    sol_key = tuple(sorted(sol))

                    if sol_key not in seen_combinations:
                        valores = [values_list[j] for j in sol]

                        # filtro opcional: must_include
                        if must_include_cents:
                            valores_cents = [values_cents[j] for j in sol]
                            if not all(mi in valores_cents for mi in must_include_cents):
                                continue

                        seen_combinations.add(sol_key)
                        found_solutions.append(sol)

                        if len(found_solutions) >= max_solutions:
                            break
        if len(found_solutions) >= max_solutions:
            break

    # retorna resultados
    results = []
    for sol in found_solutions:
        results.append([values_list[j] for j in sol])

    return results

Soluções sem filtro:
[[Decimal('2'), Decimal('3')]]

Soluções com filtro must_include:
[]


In [13]:
from decimal import Decimal

# Exemplo de lista de valores
values_list = [
    Decimal('1'), Decimal('2'), Decimal('3')
]
target = Decimal('5')

# sem filtro must_include
print("Soluções sem filtro:")
print(find_combination(values_list, target))

# com filtro must_include
print("\nSoluções com filtro must_include:")
print(find_combination(values_list, target, must_include=[Decimal('1')]))

Soluções sem filtro:
[[Decimal('2'), Decimal('3')]]

Soluções com filtro must_include:
[]
